# Running multiple calculations on a given model


## Aim

This notebook shows how we can run multiple calculations of a given structure

### Setup

The initial setup is very similar to the other tutorials, such as `singlepoint.ipynb`, which goes into more detail about what each step is doing

Load the aiida profile and code:

In [1]:
from aiida import load_profile
load_profile()

Profile<uuid='09087c8053df4044bae150654e0e6e9e' name='presto'>

In [2]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [3]:
from aiida.orm import load_code, load_computer

janus_code = load_code("janus@localhost")

computer_label = 'scarf'
scarf = load_computer(computer_label)
scarf.set_workdir('/work4/scd/scarf1480/aiida') 

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [4]:
from aiida.orm import Str, Float, Bool, Int

inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "fmax": Float(0.1), 
    "opt_cell_lengths": Bool(False), 
    "opt_cell_fully": Bool(True), 
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

We must now choose the calculations to perform:

In [5]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")


Setup graph task, which sets up QE params and created a PwCalc for each structure in file. Returns test_file, train_file and valid_file

In [6]:
from aiida_workgraph import task, WorkGraph
from aiida_workgraph.manager import get_current_graph
from aiida.orm import StructureData, load_group, KpointsData, Dict
from ase.io import read
from collections import defaultdict
from aiida_workgraph import WorkGraph
from aiida_quantumespresso.calculations.pw import PwCalculation


@task.graph(outputs = ["test_file", "train_file", "valid_file"])
def qe(**inputs):

    qe_code = load_code("qe@scarf")
    filepaths = inputs['filepath_dict']

    wg = get_current_graph()

    kpoints = KpointsData()
    kpoints.set_kpoints_mesh([1, 1, 1])

    metadata =  {
        "options": {
            "resources": {
                "num_machines": 1,
                'num_mpiprocs_per_machine': 32,
            },
            'max_wallclock_seconds': 3600,         # Set maximum wallclock time
            #'account': 'elph',                     # Set account name
            'queue_name': 'scarf',                  # Set queue name
            'qos': 'scarf',
            'environment_variables': {},
            'withmpi': True,                       # Use MPI
            'prepend_text': '''
            module purge
            module use /work4/scd/scarf562/eb-common/modules/all
            module load amd-modules
            module load QuantumESPRESSO/7.2-foss-2023a
            ''',
            'append_text': ''  
        }
    }

    pseudo_family = load_group('SSSP/1.3/PBE/efficiency')

    for _, data in filepaths.items():
            for i in range(len(read(data, index=':'))):

                structure = StructureData(ase=read(data, index=i))
                pseudos = pseudo_family.get_pseudos(structure=structure)

                ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
                    structure=structure,
                    unit='Ry',
                )

                pw_paras = {
                    "CONTROL": {
                        "calculation": "scf",
                    },
                    "SYSTEM": {
                        "ecutwfc": ecutwfc,
                        "ecutrho": ecutrho,
                    },
                }

                qe_task = wg.add_task(
                    PwCalculation,
                    code = qe_code,
                    parameters= pw_paras,
                    kpoints= kpoints,
                    pseudos= pseudos,
                    metadata= metadata,
                    structure= structure,
                )

                structfile = f"{_}.struct{i}"
                wg.update_ctx({structfile: qe_task.outputs.output_trajectory})
                
    print(wg.tasks)

    wg.outputs.test_file = wg.ctx.test_file
    wg.outputs.train_file = wg.ctx.train_file
    wg.outputs.valid_file = wg.ctx.valid_file

In [7]:
from aiida_workgraph import WorkGraph, Zone
from aiida.orm import StructureData
from ase.io import read
from sample_split import process_and_split_data
from aiida_quantumespresso.calculations.pw import PwCalculation

initial_structure = "../../examples/tutorials/structures/lj-traj.xyz"

num_structs = len(read(initial_structure, index=":"))

with WorkGraph("QE Calculation Workgraph") as wg:

    wg.inputs = inputs
    final_structures = {}

    for i in range(num_structs):
        structure = StructureData(ase=read(initial_structure, index=i))

        geomopt_calc = wg.add_task(
            geomoptCalc,
            code=wg.inputs.code,
            model=wg.inputs.model,
            arch=wg.inputs.arch,
            precision=wg.inputs.precision,
            device=wg.inputs.device,
            metadata=wg.inputs.metadata,
            fmax=wg.inputs.fmax,
            opt_cell_lengths=wg.inputs.opt_cell_lengths,
            opt_cell_fully=wg.inputs.opt_cell_fully,
            struct=structure,
        )

        descriptors_calc = wg.add_task(
            descriptorsCalc,
            code=wg.inputs.code,
            model=wg.inputs.model,
            arch=wg.inputs.arch,
            precision=wg.inputs.precision,
            device=wg.inputs.device,
            metadata=wg.inputs.metadata,
            struct=geomopt_calc.outputs.final_structure,
            calc_per_element=True,
        )

        final_structures[f"structs{i}"] = descriptors_calc.outputs.xyz_output

    split_task_inputs = {
        "trajectory_data": final_structures,
        "config_types": Str(""),
        "n_samples": Int(num_structs),
        "prefix": Str(""),
        "scale": Float(1.0e5),
        "append_mode": Bool(False),
    }

    split_task = wg.add_task(process_and_split_data, inputs=split_task_inputs)

    qe_task = wg.add_task(qe, name="QE_workflow", filepath_dict=split_task.outputs.result)


defining outputnode
defining outputnode


In [8]:
wg


NodeGraphWidget(settings={'minimap': True}, style={'width': '90%', 'height': '600px'}, value={'name': 'QE Calc…

In [9]:
wg.run()

09/04/2025 11:38:02 AM <86378> aiida.broker.rabbitmq: [WARNING] RabbitMQ v3.12.1 is not supported and will cause unexpected problems!
09/04/2025 11:38:02 AM <86378> aiida.broker.rabbitmq: [WARNING] It can cause long-running workflows to crash and jobs to be submitted multiple times.
09/04/2025 11:38:02 AM <86378> aiida.broker.rabbitmq: [WARNING] See https://github.com/aiidateam/aiida-core/wiki/RabbitMQ-version-to-use for details.
09/04/2025 11:38:21 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18270|WorkGraphEngine|continue_workgraph]: tasks ready to run: GeomOpt,GeomOpt1,GeomOpt2,GeomOpt3,GeomOpt4,GeomOpt5,GeomOpt6,GeomOpt7,GeomOpt8,GeomOpt9,GeomOpt10
09/04/2025 11:38:44 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18270|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 18274, 18278, 18282, 18286, 18290, 18294, 18298, 18302, 18306, 18310, 18314
09/04/2025 11:40:06 AM <86378> aiida.orm.nodes.proces

create files: train_file=PosixPath('train.xyz'), valid_file=PosixPath('valid.xyz') and test_file=PosixPath('test.xyz')
Processing: ('all', 'aiida'), 11 frames
  ('all', 'aiida'): total=11, train_target=8,                     vt_target=3


/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 11
  warnings.warn(f"k is too large, set to {n_pts}")
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/fpsample/wrapper.py:98: UserWarning: k is too large, set to 3
  warnings.warn(f"k is too large, set to {n_pts}")
09/04/2025 11:43:21 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18270|WorkGraphEngine|update_task_state]: Task: process_and_split_data, type: CALCFUNCTION, finished.
09/04/2025 11:43:23 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18270|WorkGraphEngine|continue_workgraph]: tasks ready to run: QE_workflow


NodeCollection(parent = "qe", nodes = ["PwCalculation", "graph_ctx", "PwCalculation1", "PwCalculation2", "PwCalculation3", "PwCalculation4", "PwCalculation5", "PwCalculation6", "PwCalculation7", "PwCalculation8", "PwCalculation9", "PwCalculation10"])


09/04/2025 11:43:33 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18270|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 18538
09/04/2025 11:43:43 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18538|WorkGraphEngine|continue_workgraph]: tasks ready to run: PwCalculation,PwCalculation1,PwCalculation2,PwCalculation3,PwCalculation4,PwCalculation5,PwCalculation6,PwCalculation7,PwCalculation8,PwCalculation9,PwCalculation10
09/04/2025 11:43:56 AM <86378> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [18538|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 18540, 18542, 18544, 18546, 18548, 18550, 18552, 18554, 18556, 18558, 18560
/home/mtr46585/aiida-mlip/.venv/lib/python3.12/site-packages/aiida/transports/plugins/ssh.py:674: AiidaDeprecationWarning: `chdir()` is deprecated and will be removed in the next major version. Use absolute paths instead. (this will be rem

In [10]:
wg.tasks.QE_workflow.outputs.train_file

SocketAny(name='train_file', value=AttributeDict({'struct0': <TrajectoryData: uuid: 1f9c769b-c775-4770-959b-8f5500c8ed1c (pk: 18574)>, 'struct5': <TrajectoryData: uuid: 952bc775-0238-4c4f-b4d5-9210133ed50f (pk: 18578)>, 'struct3': <TrajectoryData: uuid: 803c3044-efb3-4fde-9e6e-dcf9e3dc20ad (pk: 18584)>, 'struct6': <TrajectoryData: uuid: 2a9ddaed-3e53-405f-acfe-9317103fb3dc (pk: 18587)>, 'struct7': <TrajectoryData: uuid: c10688cf-57e9-4f2a-9827-aa649b023b03 (pk: 18590)>, 'struct1': <TrajectoryData: uuid: bbd6359e-7a24-45d8-ae97-e9a1e4f7ece9 (pk: 18599)>, 'struct4': <TrajectoryData: uuid: 6c010237-fad4-42e9-ab81-211e67dd6a47 (pk: 18610)>, 'struct2': <TrajectoryData: uuid: e4a7ed22-c011-436f-8523-dba3a944cb3d (pk: 18614)>}))